matrix transformation only for perfect file (with path 0)

In [13]:
import pandas as pd
import os

from positionism.read import parameter
from positionism.optimizer import opti_amount_weirdos
from positionism.plot import optimizer

direc = os.getcwd() # get current working directory

Input

In [2]:
file_perfect_poscar_48n24_wo_cif = "Li6PS5Cl_type4"
litype = 4

activate_radius = 1
max_mapping_radius = 0.043
max_mapping_radius_48htype2 = 0.066
max_mapping_radius_48htype1_48htype2 = (max_mapping_radius + max_mapping_radius_48htype2) / 2

delta_coor = 0.04
range_coor = 0.05

In [3]:
# Replace with your file path
file_perfect_poscar_48n24 = f"{file_perfect_poscar_48n24_wo_cif}.cif"

reference_folder = "_reference_cif"
results_folder = "_results"
iter_type = "varying_position"
file_path_ori_ref_48n24 = f"./{reference_folder}/{file_perfect_poscar_48n24}"
folder_results_iter_type = f"/{results_folder}/_{iter_type}/{file_perfect_poscar_48n24_wo_cif}/"

n_file_start = 0
n_file_end = 54

In [4]:
if activate_radius == 1:
    df_mapping_metainfo = pd.read_pickle(f'df_mapping_metainfo_{max_mapping_radius}_{file_perfect_poscar_48n24_wo_cif}.pkl') 
elif activate_radius == 2:
    df_mapping_metainfo = pd.read_pickle(f'df_mapping_metainfo_{max_mapping_radius}_{max_mapping_radius_48htype2}_{file_perfect_poscar_48n24_wo_cif}.pkl')
elif activate_radius == 3:
    df_mapping_metainfo = pd.read_pickle(f'df_mapping_metainfo_{max_mapping_radius}_{max_mapping_radius_48htype2}_{max_mapping_radius_48htype1_48htype2}_{file_perfect_poscar_48n24_wo_cif}.pkl')

df_wo_weirdos_selectedcol = df_mapping_metainfo[['geometry','path','subdir_new_system', 'toten [eV]']].sort_values("toten [eV]", ascending=True)[n_file_start:n_file_end].reset_index().copy()

In [5]:
max_mapping_radius_init = max_mapping_radius
max_mapping_radius_48htype2_init = max_mapping_radius_48htype2

ref_positions_array = parameter.x_y_z(file_path_ori_ref_48n24, litype)

if litype == 0:
    x, y, z = ref_positions_array[0:3]
elif litype == 1:
    x, y, z = ref_positions_array[3:6]
elif litype == 2:
    x, y, z = ref_positions_array[6:9]
elif litype == 3:
    x, y, z = ref_positions_array[9:12]
elif litype == 4:
    x, y, z = ref_positions_array[12:15]
elif litype == 5:
    x, y, z = ref_positions_array[15:18]
elif litype == 6:
    x, y, z = ref_positions_array[18:21]
elif litype == 7:
    x, y, z = ref_positions_array[21:24]
elif litype == 8:
    x, y, z = ref_positions_array[24:27]
else:
    print("litype is out of range")


# idea: outer loop dz2 positive, then dz1 ositive and negative
# 2nd loop: dz2 negative

In [6]:
min_coor_y = y - range_coor - delta_coor
max_coor_y = y + range_coor + delta_coor

min_coor_z = z - range_coor - delta_coor
max_coor_z = z + range_coor + delta_coor

In [7]:
forward_x_init = x
forward_y_init = y
forward_z_init = z

backward_x = x
backward_y = y
backward_z = z

forward_x_negative = x * -1
forward_y_negative = y * -1
forward_z_negative = z * -1

backward_x_negative = x * -1
backward_y_negative = y * -1
backward_z_negative = z * -1

In [8]:
# Define constants
col_x = "x"
col_y = "y"
col_z = "z"
col_sumweirdosLi = "sumweirdosLi"

# Initialize DataFrame
df_x_y_z_sumweirdosLi = pd.DataFrame(columns=[col_x, col_y, col_z, col_sumweirdosLi])

# Global index
idx_df = 0

# Set to track seen (x, y, z) combinations
seen_combinations = set()

# Function to update DataFrame
def update_dataframe(next_x, next_y, next_z, sum_weirdos_Li):
    global idx_df
    df_x_y_z_sumweirdosLi.at[idx_df, col_x] = next_x
    df_x_y_z_sumweirdosLi.at[idx_df, col_y] = next_y
    df_x_y_z_sumweirdosLi.at[idx_df, col_z] = next_z
    df_x_y_z_sumweirdosLi.at[idx_df, col_sumweirdosLi] = sum_weirdos_Li
    idx_df += 1

# Function to iterate over z and x
def iterate_z_x(next_y, start_z, end_z, step_z, start_x, step_x):
    next_x = start_x
    next_z = start_z
    while (step_z > 0 and next_z <= end_z) or (step_z < 0 and next_z >= end_z):
        ref_positions_array = [next_x, next_y, next_z]
        
        # Check if combination has already been processed
        if (next_x, next_y, next_z) not in seen_combinations:
            sum_weirdos_Li = opti_amount_weirdos.get_sum_weirdos_Li(
                ref_positions_array, max_mapping_radius, max_mapping_radius_48htype2, 
                df_wo_weirdos_selectedcol, activate_radius, file_perfect_poscar_48n24_wo_cif, 
                litype, iter_type
            )
            print(f"x: {next_x}, y: {next_y}, z: {next_z}, idx: {idx_df}")
            update_dataframe(next_x, next_y, next_z, sum_weirdos_Li)
            seen_combinations.add((next_x, next_y, next_z))

        next_z += step_z
        next_x += step_x

# Main loop function
def main_loop():
    global next_y
    # Loop 1: All forward
    next_y = y
    while next_y <= max_coor_y:
        iterate_z_x(next_y, z, max_coor_z, delta_coor, x, -delta_coor)
        next_y += delta_coor

    # Loop 2: Inner loop backward
    next_y = y
    while next_y <= max_coor_y:
        iterate_z_x(next_y, z, min_coor_z, -delta_coor, x, delta_coor)
        next_y += delta_coor

    # Loop 3: All backward
    next_y = y
    while next_y >= min_coor_y:
        iterate_z_x(next_y, z, min_coor_z, -delta_coor, x, delta_coor)
        next_y -= delta_coor

    # Loop 4: Outer backward
    next_y = y
    while next_y >= min_coor_y:
        iterate_z_x(next_y, z, max_coor_z, delta_coor, x, -delta_coor)
        next_y -= delta_coor


In [9]:
# Run the main loop
main_loop()

# Display the resulting DataFrame
print(df_x_y_z_sumweirdosLi)

/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

x: 0.78917, y: 0.00875, z: 0.21083, idx: 0


/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

x: 0.74917, y: 0.00875, z: 0.25083, idx: 1


/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

...

x: 0.70917, y: -0.03125, z: 0.29083, idx: 22


/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

x: 0.74917, y: -0.07125000000000001, z: 0.25083, idx: 23


/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

x: 0.70917, y: -0.07125000000000001, z: 0.29083, idx: 24
          x        y        z sumweirdosLi
0   0.78917  0.00875  0.21083         30.0
1   0.74917  0.00875  0.25083         40.0
2   0.70917  0.00875  0.29083         30.0
3   0.78917  0.04875  0.21083         38.0
4   0.74917  0.04875  0.25083         46.0
5   0.70917  0.04875  0.29083         40.0
6   0.78917  0.08875  0.21083         46.0
7   0.74917  0.08875  0.25083         46.0
8   0.70917  0.08875  0.29083         46.0
9   0.82917  0.00875  0.17083         46.0
10  0.86917  0.00875  0.13083         46.0
11  0.82917  0.04875  0.17083         44.0
12  0.86917  0.04875  0.13083         44.0
13  0.82917  0.08875  0.17083         44.0
14  0.86917  0.08875  0.13083         44.0
15  0.78917 -0.03125  0.21083         42.0
16  0.82917 -0.03125  0.17083         46.0
17  0.86917 -0.03125  0.13083         46.0
18  0.78917 -0.07125  0.21083         46.0
19  0.82917 -0.07125  0.17083         46.0
20  0.86917 -0.07125  0.13083         46

In [10]:
min_sumweirdos = int(df_x_y_z_sumweirdosLi["sumweirdosLi"].min())

In [11]:
if activate_radius == 2:
    df_x_y_z_sumweirdosLi.to_excel(f'08_varying_dx_dz_{min_sumweirdos}_{max_mapping_radius}_{max_mapping_radius_48htype2}_{file_perfect_poscar_48n24_wo_cif}.xlsx', index=False)
elif activate_radius == 1:
    df_x_y_z_sumweirdosLi.to_excel(f'08_varying_dx_dz_{min_sumweirdos}_{max_mapping_radius}_{file_perfect_poscar_48n24_wo_cif}.xlsx', index=False)

In [15]:
%matplotlib inline

optimizer.heatmap_x_y(df_x_y_z_sumweirdosLi, folder_results_iter_type)

AttributeError: module 'positionism.plot.optimizer' has no attribute 'heatmap_dx_dz'